In [1]:
dtypes = {
    'comment_text':'str',
    'comment_id':'str',
    'page_id': 'str',
    'parent_id': 'str', 
    'post_id': 'str',
    'comment_reaction_count':'float',
    'comment_reply_count':'float',
    'comment_reaction1':'str',
    'comment_reaction2':'str',
    'comment_reaction3':'str',
    'comment_topfan':'float',
    'like':'float',
    'super_like':'float',
    'sup_like':'float',
    'angry':'float',
    'super_angry':'float',
    'sup_angry':'float',
    'love':'float',
    'super_love':'float',
    'sup_love':'float',
    'haha':'float',
    'super_haha':'float',
    'sup_haha':'float',
    'sad':'float',
    'super_sad':'float',
    'sup_sad':'float',
    'wow':'float',
    'super_wow':'float',
    'sup_wow':'float',
    'is_skoi':'float',
    'thread_level': 'float',
    'post_image': 'float',
    'post_video': 'float',
    'post_comment_count':'float',
    'post_share_count':'float',
    'post_reaction_count':'float',
    'post_like': 'float',
    'post_love': 'float',
    'post_haha': 'float', 
    'post_sad': 'float',
    'post_wow': 'float',
    'post_angry': 'float',
    'comment_image':'float',
    'comment_video':'float',
    'comment_year':'float',
    'comment_month': 'float',
    'comment_day': 'float',
    'comment_hour': 'float',
    'comment_minute': 'float',
    'comment_second': 'float',
    'comment_day_of_week': 'float',
    'post_year':'float',
    'post_month':'float',
    'post_day':'float',
    'post_hour': 'float',
    'post_minute': 'float',
    'post_second': 'float',
    'post_day_of_week': 'float',
    'comment_text_char_len': 'float',
    'comment_num_hashtag': 'float',
    'comment_avg_char_hashtag':'float',
    'comment_urls_len':'float',
    'comment_neg_emo_count':'float',
    'comment_neu_emo_count': 'float',
    'comment_pos_emo_count': 'float',
    'comment_author': 'str'   
}
context_features = [
    'comment_reaction_count','comment_reply_count', 'comment_reaction1','comment_reaction2','comment_reaction3',
    'comment_topfan','like','super_like','sup_like','angry','super_angry','sup_angry','love','super_love','sup_love',
    'haha','super_haha','sup_haha','sad','super_sad','sup_sad','wow','super_wow','sup_wow','is_skoi',
    'thread_level','post_image','post_video','post_comment_count','post_share_count','post_reaction_count',
    'post_like','post_love','post_haha','post_sad','post_wow','post_angry',
    'comment_image','comment_video','comment_year','comment_month','comment_day','comment_hour',
    'comment_minute','comment_second','comment_day_of_week','post_year','post_month',
    'post_day','post_hour','post_minute','post_second','post_day_of_week','comment_text_char_len',
    'comment_num_hashtag','comment_avg_char_hashtag','comment_urls_len','comment_neg_emo_count',
    'comment_neu_emo_count','comment_pos_emo_count']

content_features = ['comment_text']
labels = ['rude','figurative','offensive','dirty']


reaction_map = {
    'haha': 1,
    'like': 2,
    'angry': 3,
    'sad': 4,
    'wow': 5,
    'love': 6,
    '<NONE>': 7
}

In [2]:
import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from tqdm import tqdm_notebook
import re
import emoji

#viz
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval
from collections import Counter

#fastai
import fastai
from fastai.text import *
from fastai.callbacks import CSVLogger

#pythainlp
from pythainlp.ulmfit import *


import numpy as np
from sklearn.model_selection import train_test_split

from fastai.text import *
from fastai.callbacks import CSVLogger, SaveModelCallback
from pythainlp.ulmfit import *


from sklearn.metrics import f1_score

import time



from sklearn.metrics import confusion_matrix

In [3]:
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def process_text(text):
    #pre rules
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)
    
    #tokenize
    res = [word for word in word_tokenize(res, engine='ulmfit') if word and not re.search(pattern=r"\s+", string=word)]
    
    #post rules
    res = ungroup_emoji(res)
    
    return res

In [4]:
def train_df(all_df, abuss_type):
    
    train_df, valid_df = train_test_split(all_df, test_size=0.15)
    train_df = train_df.reset_index(drop=True)
    valid_df = valid_df.reset_index(drop=True)


    y_train = train_df[abuss_type]
    y_valid = valid_df[abuss_type]

    model_path = f'model_ulm_{abuss_type}/'

    train_df, valid_df = train_test_split(all_df, test_size=0.15)

    tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)

    processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
              NumericalizeProcessor(vocab=None, max_vocab=50000, min_freq=2)]

    data_lm = (TextList.from_df(all_df, model_path, cols='comment_text', processor=processor)
      .random_split_by_pct(valid_pct = 0.01, seed = 1412)
      .label_for_lm()
      .databunch(bs=48))

    data_lm.sanity_check()

    len(data_lm.train_ds), len(data_lm.valid_ds)

    config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
              output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)

    trn_args = dict(drop_mult=1., clip=0.12, alpha=2, beta=1)

    learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args)

    learn.load_pretrained(**_THWIKI_LSTM)

    #train frozen
    print('training frozen')
    learn.freeze_to(-1)#last layer
    learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

    #train unfrozen
    print('training unfrozen')
    learn.unfreeze()
    learn.fit_one_cycle(5, 1e-3, moms=(0.8, 0.7))

    learn.model


    learn.save('wisesight_lm')
    learn.save_encoder('wisesight_enc')

    #classification data
    tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)
    processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
              NumericalizeProcessor(vocab=data_lm.vocab, max_vocab=50000, min_freq=20)]

    data_cls = (ItemLists(model_path,train=TextList.from_df(train_df, model_path, cols=['comment_text'], processor=processor),
                      valid=TextList.from_df(valid_df, model_path, cols=['comment_text'], processor=processor))
      .label_from_df(abuss_type)
      .databunch(bs=50)
      )
    data_cls.sanity_check()
    print(len(data_cls.vocab.itos))

    #model
    config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False,
              output_p=0.4, hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5)
    trn_args = dict(bptt=70, drop_mult=0.7, alpha=2, beta=1, max_len=500)

    learn = text_classifier_learner(data_cls, AWD_LSTM, config=config, pretrained=False, **trn_args)
    #load pretrained finetuned model
    learn.load_encoder('wisesight_enc')

    #train unfrozen
    learn.freeze_to(-1)
    learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))
    learn.freeze_to(-2)
    learn.fit_one_cycle(1, slice(1e-2 / (2.6 ** 4), 1e-2), moms=(0.8, 0.7))
    learn.freeze_to(-3)
    learn.fit_one_cycle(1, slice(5e-3 / (2.6 ** 4), 5e-3), moms=(0.8, 0.7))
    learn.unfreeze()
    learn.fit_one_cycle(10, slice(1e-3 / (2.6 ** 4), 1e-3), moms=(0.8, 0.7), callbacks=[SaveModelCallback(learn, every='improvement', monitor='accuracy', name='bestmodel')])
    
    return learn

def embedding(s, learn):
    sentence = learn.model[0](learn.data.one_item(s)[0])[0][-1][:, -1]
    return sentence.cpu().detach().numpy()

def embedding_df(df, learn):
    for i in range(400):
        df['sent_emb_' + str(i)] = None
    a = np.zeros((len(df),400))
    for i, v in enumerate(df.comment_text):
        a[i] = embedding(v, learn)
    df.iloc[:,df.columns.get_loc('sent_emb_0'):] = a
    return df

def predict_df(df, learn, abuss_type, show = False):
    model_path = f'model_ulm_{abuss_type}/'
    tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)
    processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
              NumericalizeProcessor(vocab=None, max_vocab=50000, min_freq=2)]
    learn.data.add_test(TextList.from_df(df, model_path, cols=['comment_text'], processor=processor))
    tmp = learn.get_preds(DatasetType.Test)
    if show:
        df['pred'] = tmp[0].numpy()[:, 1]
#         df.pred = df.pred.apply(round)
#         print(f"{abuss_type}: f1 = {f1_score(df[abuss_type], df.pred)}")
#         print(pd.crosstab(df[abuss_type], df.pred, rownames=['Actual'], colnames=['Predicted'], margins=True))
    
    return df.pred

In [5]:
def pre_df(df3):
    df3.rude = df3.rude.apply(int)
    df3.figurative = df3.figurative.apply(int)
    df3.offensive = df3.offensive.apply(int)
    df3.dirty = df3.dirty.apply(int)
    return df3

In [6]:
Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/dirty_vs_non.csv', dtype=dtypes)
Unlabeled_Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/unlabeled_data_temp1.csv', dtype=dtypes)

In [7]:
#remove duplicate
label_id_set = set(Corpus['comment_id'])
unlabel_id_set = set(Unlabeled_Corpus['comment_id'])
duplicate_comment_id_set = label_id_set.intersection(unlabel_id_set)
Unlabeled_Corpus = Unlabeled_Corpus.loc[~Unlabeled_Corpus['comment_id'].isin(duplicate_comment_id_set)]
Unlabeled_Corpus.reset_index(inplace=True)


In [8]:
reaction_columns = [f'comment_reaction{i}' for i in range(1, 4)]
Corpus[reaction_columns] = Corpus[reaction_columns].replace(reaction_map)
Unlabeled_Corpus[reaction_columns] = Unlabeled_Corpus[reaction_columns].replace(reaction_map)

In [9]:
Corpus = pre_df(Corpus)
Train_X, Test_X = train_test_split(Corpus, train_size = 0.8)

In [10]:
Train_Y = Train_X[['rude','figurative','offensive','dirty']]
Test_Y = Test_X[['rude','figurative','offensive','dirty']]

In [11]:
learn = train_df(Train_X, 'dirty')

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/fastai/data_block.py:215: UserWarning: `random_split_by_pct` is deprecated, please use `split_by_rand_pct`.
  warn("`random_split_by_pct` is deprecated, please use `split_by_rand_pct`.")


training frozen


epoch,train_loss,valid_loss,accuracy,time
0,5.152982,4.308551,0.366071,00:08


training unfrozen


epoch,train_loss,valid_loss,accuracy,time
0,4.552886,4.109659,0.380060,00:08
1,4.374611,3.990049,0.383333,00:08
2,4.192266,3.921504,0.386310,00:08
3,4.043765,3.902444,0.391369,00:08
4,3.946097,3.899783,0.388095,00:08


...

epoch,train_loss,valid_loss,accuracy,time
0,0.120503,0.098904,0.967905,00:11


epoch,train_loss,valid_loss,accuracy,time
0,0.111323,0.091143,0.969739,00:10


epoch,train_loss,valid_loss,accuracy,time
0,0.091173,0.072787,0.976158,00:13


epoch,train_loss,valid_loss,accuracy,time
0,0.073317,0.072363,0.976616,00:12
1,0.076982,0.072570,0.977075,00:13
2,0.064468,0.072926,0.977992,00:13
3,0.063533,0.072989,0.977075,00:13
4,0.048902,0.077450,0.977075,00:13
5,0.044252,0.080672,0.977533,00:12
6,0.038023,0.078738,0.977533,00:13
7,0.042752,0.080164,0.977075,00:13
8,0.035440,0.081810,0.978450,00:13
9,0.028594,0.083582,0.977992,00:13


Better model found at epoch 0 with accuracy value: 0.9766162037849426.
Better model found at epoch 1 with accuracy value: 0.9770747423171997.
Better model found at epoch 2 with accuracy value: 0.9779917597770691.
Better model found at epoch 8 with accuracy value: 0.9784502387046814.


In [12]:
from scipy.sparse import hstack, csr_matrix, vstack
train_feat_context = Train_X[context_features].values
test_feat_context = Test_X[context_features].values
unlabeled_feat_context = Unlabeled_Corpus[context_features].values

In [13]:
def createGeneralEvaluation(file):
    f = open(file, mode='w')
    f.write("time,amount_of_data,data_type,threshold,type_class,accuracy,precision,recall,fscore,support,auc_roc,auc_pr")
    f.write("\n")
    f.close()

In [14]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
def report_performance(file,time,modelType,predictions_class_train_c, predictions_class_test_c, type_class,threshold, predict_train_prob,predict_test_prob, train_label,train_feat):
    #Evaluation class (test data)
    print("Test data of "+modelType)
    data_type_test = "Test data of "+modelType
    threshold = str(threshold)
    type_class = str(type_class)
    accuracy_test = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    precision,recall,fscore,support=score(Test_Y[type_class],predictions_class_test_c)
    precision_test = (format(precision[1]))
    recall_test = (format(recall[1]))
    fscore_test = (format(fscore[1]))
    support_test = (format(support[1]))
    auc_roc_test = str(roc_auc_score(Test_Y[type_class], predict_test_prob))
    print("PRF: \n", classification_report(Test_Y[type_class],predictions_class_test_c))
    print("Area under the curve ROC: ", roc_auc_score(Test_Y[type_class], predict_test_prob))
    lr_precision_test, lr_recall_test, _ = precision_recall_curve(Test_Y[type_class], predict_test_prob)
    auc_pr_test = str(auc(lr_recall_test, lr_precision_test))
    print("Area under the curve PR: ", auc(lr_recall_test, lr_precision_test))
    print("-----------------------------------------------------------\n")

    #Evaluation class (train data)
    print("Train data of "+modelType)
    data_type_train = "Train data of "+modelType
    accuracy_train = str(accuracy_score(predictions_class_train_c, train_label))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_train_c, train_label)*100)
    print("Confusion Matrix: \n",confusion_matrix(train_label,predictions_class_train_c))
    precision,recall,fscore,support=score(train_label,predictions_class_train_c)
    precision_train = (format(precision[1]))
    recall_train  = (format(recall[1]))
    fscore_train  = (format(fscore[1]))
    support_train  = (format(support[1]))
    auc_roc_train  = str(roc_auc_score(train_label, predict_train_prob))
    print("PRF: \n",classification_report(train_label,predictions_class_train_c))
    print("Area under the curve ROC: ", roc_auc_score(train_label, predict_train_prob))
    lr_precision_train, lr_recall_train, _ = precision_recall_curve(train_label, predict_train_prob)
    auc_pr_train  = str(auc(lr_recall_train, lr_precision_train))
    print("Area under the curve PR: ", auc(lr_recall_train, lr_precision_train))
    print("___________________________________________________________________________________________________________\n")
    
    f = open(file, mode='a')
    f.write(str(time)+","+str(train_feat)+","+data_type_test+","+threshold+","+type_class+","+accuracy_test+","+precision_test+","+recall_test+","+fscore_test+","+support_test+","+auc_roc_test+","+auc_pr_test)
    f.write("\n")
    f.write(str(time)+","+str(train_feat)+","+data_type_train+","+threshold+","+type_class+","+accuracy_train+","+precision_train+","+recall_train+","+fscore_train+","+support_train+","+auc_roc_train+","+auc_pr_train)
    f.write("\n")
    
    f.close()

In [15]:
import sklearn.ensemble as ensemble
import numpy
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
    
def train_and_label_context(train_X, train_y, test_X, test_y, u_X, file, type_class, time, dataType):
    model = RandomForestClassifier(n_estimators=20, n_jobs=20)
    model.fit(train_X, train_y)
    
    # eval
    predictions_test = model.predict_proba(test_X)
    predictions_train = model.predict_proba(train_X)
    for th in numpy.arange(0.1,1.1,0.1):
        predictions_class_Train = (predictions_train[:,1] >= th).astype('int')
        predictions_class_Test = (predictions_test[:,1] >= th).astype('int')

        report_performance(file,time,dataType,predictions_class_Train,predictions_class_Test, type_class,th, predictions_train[:,1], predictions_test[:,1],train_y,train_X.shape[0])

    # add new data
    
    predictions_unlabeled = model.predict_proba(u_X)
    certain_positives = np.argwhere(predictions_unlabeled[:, 1] > 0.9).reshape(-1)
    certain_negatives = np.argwhere(predictions_unlabeled[:, 1] < 0.1).reshape(-1)
    return certain_positives, certain_negatives

Using TensorFlow backend.


In [16]:
import sklearn.ensemble as ensemble
import numpy
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
    
def train_and_label_content(model,train_X, test_X, u_X, file, type_class, time, dataType):
    
    # eval
    predictions_test = predict_df(test_X, model, type_class, show = True)
    predictions_train = predict_df(train_X, model, type_class, show = True)
    for th in numpy.arange(0.1,1.1,0.1):
        predictions_class_Train = (predictions_train >= th).astype('int')
        predictions_class_Test = (predictions_test >= th).astype('int')

        report_performance(file,time,dataType,predictions_class_Train,predictions_class_Test, type_class,th, predictions_train, predictions_test,train_X[type_class],train_X.shape[0])

    # add new data
    
    predictions_unlabeled = predict_df(u_X, model, type_class, show = True)
    certain_positives = np.argwhere(predictions_unlabeled > 0.9).reshape(-1)
    certain_negatives = np.argwhere(predictions_unlabeled < 0.1).reshape(-1)
    return certain_positives, certain_negatives

In [17]:
file = 'report_uml_dirty_add5000pertime.csv'
createGeneralEvaluation(file)

In [18]:
from scipy.sparse import hstack, csr_matrix, vstack
label = 'dirty'
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
content_train_y = _train_y
context_train_y = _train_y
train_feat_content = Train_X

content_certain_set = set()
content_certain_labels = pd.DataFrame(index=Unlabeled_Corpus.index)
content_certain_labels[label] = -1

context_certain_set = set()
context_certain_labels = pd.DataFrame(index=Unlabeled_Corpus.index)
context_certain_labels[label] = -1

for i in range(1,11):
    time = i
    # certain data from content
    _cp, _cn = train_and_label_content(learn,
        train_feat_content, 
        Test_X,
        Unlabeled_Corpus, file, label, time, "content")

    # certain data from context
    _xp, _xn = train_and_label_context(
        train_feat_context, context_train_y, 
        test_feat_context, _test_y,
        unlabeled_feat_context, file, label, time, "context")

    # Update certain data from content to context model
    content_certain_set.update(list(_cp))
    content_certain_set.update(list(_cn))
    content_certain_labels.loc[_cp, label] = 1
    content_certain_labels.loc[_cn, label] = 0

    _ccl = list(content_certain_set)
    start = 5000*(time-1)
    end = 5000*(time)
    _ccl = _ccl[start:end]
    _cc_X = unlabeled_feat_context[_ccl]
    _cc_y = content_certain_labels.loc[_ccl, label]

    train_feat_context = vstack([train_feat_context, _cc_X])
    context_train_y = np.concatenate([context_train_y, _cc_y.values], axis=0)
    
    print('current amount of data in context model'+str(train_feat_context.shape))
    
    # Update certain data from context to content model
    context_certain_set.update(list(_xp))
    context_certain_set.update(list(_xn))
    context_certain_labels.loc[_xp, label] = 1
    context_certain_labels.loc[_xn, label] = 0

    _xxl = list(context_certain_set)
    _xxl = _xxl[start:end]
    _xx_X = Unlabeled_Corpus.iloc[_xxl]
    _xx_y = context_certain_labels.loc[_xxl, label]

    certain_data = _xx_X
    certain_data[label] = _xx_y
    train_feat_content = train_feat_content.append(certain_data)
    
    #train model content
    print('current amount of data in content model'+str(train_feat_content.shape))
    learn = train_df(train_feat_content, label)

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/ipykernel_launcher.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/ipykernel_launcher.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Test data of content
dirty Accuracy score: 96.5327462850853
Confusion Matrix: 
 [[3426   83]
 [  43   82]]
PRF: 
               precision    recall  f1-score   support

           0       0.99      0.98      0.98      3509
           1       0.50      0.66      0.57       125

    accuracy                           0.97      3634
   macro avg       0.74      0.82      0.77      3634
weighted avg       0.97      0.97      0.97      3634

Area under the curve ROC:  0.9108349957252779
Area under the curve PR:  0.5501062311577304
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 98.82352941176471
Confusion Matrix: 
 [[13927   138]
 [   33   437]]
PRF: 
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     14065
           1       0.76      0.93      0.84       470

    accuracy                           0.99     14535
   macro avg       0.88      0.96      0.92     14535
weighted avg   

PRF: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99     14065
           1       0.98      0.67      0.80       470

    accuracy                           0.99     14535
   macro avg       0.98      0.84      0.90     14535
weighted avg       0.99      0.99      0.99     14535

Area under the curve ROC:  0.9935956917351809
Area under the curve PR:  0.9394444570156615
___________________________________________________________________________________________________________

Test data of content
dirty Accuracy score: 96.56026417171161
Confusion Matrix: 
 [[3509    0]
 [ 125    0]]
PRF: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3509
           1       0.00      0.00      0.00       125

    accuracy                           0.97      3634
   macro avg       0.48      0.50      0.49      3634
weighted avg       0.93      0.97      0.95      3634

Area under the curve ROC

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

...

epoch,train_loss,valid_loss,accuracy,time
0,0.037366,0.029471,0.992769,00:54


epoch,train_loss,valid_loss,accuracy,time
0,0.023877,0.022297,0.993906,01:01


epoch,train_loss,valid_loss,accuracy,time
0,0.023346,0.028753,0.993596,00:57


epoch,train_loss,valid_loss,accuracy,time
0,0.027613,0.027101,0.993802,01:08
1,0.018375,0.028744,0.993492,00:59
2,0.017312,0.030538,0.994009,01:00
3,0.020023,0.034901,0.993389,00:57
4,0.013118,0.027439,0.994112,01:00
5,0.010083,0.033999,0.993286,01:08
6,0.009622,0.032045,0.994112,00:59
7,0.004933,0.033810,0.993802,00:59
8,0.005480,0.033757,0.993596,00:59
9,0.009253,0.033586,0.993802,00:59


Better model found at epoch 0 with accuracy value: 0.9938023090362549.
Better model found at epoch 2 with accuracy value: 0.9940088987350464.
Better model found at epoch 4 with accuracy value: 0.9941121935844421.
